In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# CODE CELL 1: SETUP & CONFIGURATION

# --- 1. INSTALL ALL DEPENDENCIES ---
print("Installing all required libraries...")
!pip install ultralytics opencv-python shapely deep-sort-realtime -q
print("✅ Installation complete.")

# --- 2. DATA UNIFICATION (WITH THE CORRECTED FUNCTION) ---
import os
import glob
import shutil
import yaml
print("\n--- Starting Multi-Class Data Unification ---")

# This is the corrected, readable, and functional version
def process_and_copy_files(source_dir, dest_img_dir, dest_lbl_dir, prefix, source_map):
    # First, check if the main source directory exists.
    if not os.path.exists(source_dir):
        print(f"⚠️ Warning: Source directory not found: {source_dir}. Skipping.")
        return

    # Define the paths for images and labels subdirectories.
    # This now runs correctly after the check above passes.
    img_path = os.path.join(source_dir, 'images')
    lbl_path = os.path.join(source_dir, 'labels')

    # Second, check if the essential 'images' subdirectory exists.
    if not os.path.exists(img_path):
        print(f"⚠️ Warning: Images subdirectory not found in {source_dir}. Skipping.")
        return

    # Proceed with file processing
    image_files = glob.glob(os.path.join(img_path, '*.jpg')) + glob.glob(os.path.join(img_path, '*.png'))
    print(f"Processing {len(image_files)} files from {source_dir}...")
    for img_file in image_files:
        name, _ = os.path.splitext(os.path.basename(img_file))
        lbl_file = os.path.join(lbl_path, name + '.txt')
        dest_img_file = os.path.join(dest_img_dir, prefix + name + os.path.splitext(img_file)[1])
        dest_lbl_file = os.path.join(dest_lbl_dir, prefix + name + '.txt')
        shutil.copy(img_file, dest_img_file)
        if os.path.exists(lbl_file):
            with open(lbl_file, 'r') as f_in, open(dest_lbl_file, 'w') as f_out:
                for line in f_in:
                    parts = line.strip().split()
                    if len(parts) != 5: continue
                    original_id = int(parts[0]); class_name = source_map.get(original_id)
                    if class_name in FINAL_CLASS_IDS: new_id = FINAL_CLASS_IDS[class_name]; parts[0] = str(new_id); f_out.write(' '.join(parts) + '\n')

FINAL_CLASS_NAMES=['car','truck','bus','motorcycle']
CLASS_MAP={'traffic_wala':{0:'car',1:'motorcycle',2:'truck',3:'bus'},'bdd':{1:'car',2:'truck',3:'bus',4:'motorcycle'}}
FINAL_CLASS_IDS={name:i for i,name in enumerate(FINAL_CLASS_NAMES)}
source_1_path='/kaggle/input/traffic-dataset/traffic_wala_dataset/'
output_path='/kaggle/working/merged_data/'
train_img_out,train_lbl_out=os.path.join(output_path,'train/images/'),os.path.join(output_path,'train/labels/')
valid_img_out,valid_lbl_out=os.path.join(output_path,'valid/images/'),os.path.join(output_path,'valid/labels/')
os.makedirs(train_img_out,exist_ok=True);os.makedirs(train_lbl_out,exist_ok=True);os.makedirs(valid_img_out,exist_ok=True);os.makedirs(valid_lbl_out,exist_ok=True)
process_and_copy_files(os.path.join(source_1_path,'train'), train_img_out, train_lbl_out, 'traffic_', CLASS_MAP['traffic_wala'])
process_and_copy_files(os.path.join(source_1_path,'valid'), valid_img_out, valid_lbl_out, 'traffic_', CLASS_MAP['traffic_wala'])
print("--- Data Unification Complete ---")

# --- 3. CREATE CONFIG FILES ---
data_yaml_content={'train':'/kaggle/working/merged_data/train/images','val':'/kaggle/working/merged_data/valid/images','nc':len(FINAL_CLASS_NAMES),'names':FINAL_CLASS_NAMES}
DATA_YAML_PATH = '/kaggle/working/vehicle_data.yaml'
with open(DATA_YAML_PATH, 'w') as f: yaml.dump(data_yaml_content, f, default_flow_style=False)
print(f"✅ Created data config at: {DATA_YAML_PATH}")
CUSTOM_MODEL_YAML_CONTENT = f"""
nc: {len(FINAL_CLASS_NAMES)}
depth_multiple: 1.00
width_multiple: 1.00
backbone:
  - [-1, 1, Conv, [64, 3, 2]]
  - [-1, 1, Conv, [128, 3, 2]]
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]]
  - [-1, 6, C2f, [256, True]]
  - [-1, 1, Conv, [512, 3, 2]]
  - [-1, 6, C2f, [512, True]]
  - [-1, 1, Conv, [1024, 3, 2]]
  - [-1, 3, C2f, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]]
head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 6], 1, Concat, [1]]
  - [-1, 3, C2f, [512]]
  - [-1, 1, nn.Dropout, [0.1]]
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 4], 1, Concat, [1]]
  - [-1, 3, C2f, [256]]
  - [-1, 1, nn.Dropout, [0.1]]
  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 12], 1, Concat, [1]]
  - [-1, 3, C2f, [512]]
  - [-1, 1, nn.Dropout, [0.1]]
  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 9], 1, Concat, [1]]
  - [-1, 3, C2f, [1024]]
  - [-1, 1, nn.Dropout, [0.1]]
  - [[16, 20, 24], 1, Detect, [nc]]
"""
CUSTOM_MODEL_YAML_PATH = '/kaggle/working/yolov8l_dropout.yaml'
with open(CUSTOM_MODEL_YAML_PATH, 'w') as f: f.write(CUSTOM_MODEL_YAML_CONTENT)
print(f"✅ Created STABLE custom model config at: {CUSTOM_MODEL_YAML_PATH}")

print("\n\n\n---")
print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
print("!!! CRITICAL: SETUP COMPLETE. Please restart the kernel NOW.           !!!")
print("!!! Use the 'Run' menu -> 'Restart Session' or the circular arrow icon.!!!")
print("!!! After restarting, DO NOT run this cell again. Run Cell 2 directly. !!!")
print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

In [ ]:
# CODE CELL 2: TRAINING & ANALYSIS

# --- 1. IMPORTS ---
# This will now work because the kernel was restarted after installation.
import cv2
import torch
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from shapely.geometry import Point, Polygon
print("✅ Imports successful. The ultralytics library was found.")


# --- 2. TRAINING ---
print("\n--- Starting Model Training ---")
CUSTOM_MODEL_YAML_PATH = '/kaggle/working/yolov8l_dropout.yaml'
DATA_YAML_PATH = '/kaggle/working/vehicle_data.yaml'

model = YOLO(CUSTOM_MODEL_YAML_PATH)
model.train(
    data=DATA_YAML_PATH,
    pretrained='yolov8l.pt',
    imgsz=640,
    batch=8,
    epochs=300,
    name='yolov8l_dropout_run',
    project='/kaggle/working/runs/train',
    cache=True,
    patience=30,
)
print(f"\n--- Training Complete ---")



In [ ]:
# --- 3. ANALYSIS ---
print("\n--- Starting Video Analysis ---")

# Import necessary libraries if not already imported
import torch
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from shapely.geometry import Point, Polygon

BEST_MODEL_PATH = '/kaggle/working/runs/train/yolov8l_dropout_run/weights/best.pt'
FINAL_CLASS_NAMES = ['car', 'truck', 'bus', 'motorcycle']
VIDEO_PATH = '/kaggle/input/videohai/traffic.mp4'

# --- FIX 2: Correct the output path to the writable /kaggle/working/ directory ---
OUTPUT_VIDEO_PATH = '/kaggle/working/traffic_analysis_output.mp4'

LANE_LINES = [[(100,720),(500,450)],[(550,720),(650,450)],[(800,720),(850,450)],[(1200,720),(1100,450)]]
CONF_THRESHOLD = 0.4

device='cuda' if torch.cuda.is_available() else 'cpu'
model=YOLO(BEST_MODEL_PATH); model.to(device)
tracker=DeepSort(max_age=30,nn_budget=50,n_init=3)
cap=cv2.VideoCapture(VIDEO_PATH)

if not cap.isOpened(): 
    print(f"Error opening video {VIDEO_PATH}")
else:
    w,h,fps=(int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    out=cv2.VideoWriter(OUTPUT_VIDEO_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w,h))
    
    # Setup lane polygons and data structures
    lane_polygons=[Polygon(LANE_LINES[i]+LANE_LINES[i+1][::-1]) for i in range(len(LANE_LINES)-1)]
    lane_vehicle_ids={i+1:{cls:set() for cls in FINAL_CLASS_NAMES} for i in range(len(lane_polygons))}
    
    frame_num=0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Analyzing video with {total_frames} frames...")
    
    while cap.isOpened():
        ret,frame=cap.read()
        if not ret:break
            
        if frame_num % 100 == 0:
            print(f"Processing frame {frame_num}/{total_frames}...")
        frame_num+=1
        
        # Get detections from YOLOv8
        results=model(frame, verbose=False)
        
        # Prepare detections for DeepSort in [l,t,w,h] format
        detections=[]
        for box in results[0].boxes:
            if box.conf[0] > CONF_THRESHOLD:
                x1,y1,x2,y2 = box.xyxy[0].cpu().numpy()
                w_box, h_box = x2 - x1, y2 - y1
                detections.append(([int(x1), int(y1), int(w_box), int(h_box)], box.conf[0], int(box.cls[0])))
        
        # Update the tracker
        tracks=tracker.update_tracks(detections, frame=frame)
        
        # Process each track
        for track in tracks:
            if not track.is_confirmed() or track.time_since_update > 2:
                continue
                
            # --- FIX 1: Removed the 'orig_image_wh' argument from to_ltrb() ---
            track_id, bbox, cls_id = track.track_id, track.to_ltrb(), track.get_det_class()
            
            class_name = FINAL_CLASS_NAMES[cls_id]
            
            # Check which lane the object is in
            # Using the bottom-center point of the bounding box for lane assignment
            bottom_center_point = Point((bbox[0] + bbox[2]) / 2, bbox[3])
            assigned_lane = next((i + 1 for i, poly in enumerate(lane_polygons) if poly.contains(bottom_center_point)), None)
            
            if assigned_lane:
                lane_vehicle_ids[assigned_lane][class_name].add(track_id)
            
            # Draw bounding box and label
            label = f"{class_name} {track_id}" + (f" L:{assigned_lane}" if assigned_lane else "")
            cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 255, 0), 2)
            cv2.putText(frame, label, (int(bbox[0]), int(bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            
        # Draw lane counts on the frame
        y_offset = 40
        for lane_num, class_counts in lane_vehicle_ids.items():
            total_in_lane = sum(len(ids) for ids in class_counts.values())
            cv2.putText(frame, f"Lane {lane_num} Total: {total_in_lane}", (20, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3)
            y_offset += 30
            breakdown = ", ".join([f"{name}:{len(ids)}" for name, ids in class_counts.items() if len(ids) > 0])
            cv2.putText(frame, breakdown, (30, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 2)
            y_offset += 40
            
        out.write(frame)
        
    cap.release()
    out.release()
    
    print(f"\n--- Analysis Complete ---")
    print(f"Output video saved to: {OUTPUT_VIDEO_PATH}")

In [ ]:
# --- IMPORTS ---
import torch
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from shapely.geometry import Point, Polygon
import time
from collections import defaultdict

# --- CONFIGURATION ---
print("\n--- Initializing Configuration ---")

# Model and Video Paths
BEST_MODEL_PATH = '/kaggle/working/runs/train/yolov8l_dropout_run/weights/best.pt'
VIDEO_PATH = '/kaggle/input/videohai/traffic.mp4'
OUTPUT_VIDEO_PATH = '/kaggle/working/traffic_analysis_stable.mp4'

# --- DETECTION & TRACKING PARAMETERS (FOR STABILITY) ---
TARGET_CLASSES = [0, 1, 2, 3] # 'car', 'truck', 'bus', 'motorcycle'
CLASS_NAMES = ['car', 'truck', 'bus', 'motorcycle']

# STABILITY FIX 1: Lower the confidence threshold to allow weaker detections to be processed by the tracker.
# The tracker's logic is better at filtering noise than a simple confidence score.
CONF_THRESHOLD = 0.3
IOU_THRESHOLD = 0.5

# STABILITY FIX 2: Tune DeepSORT parameters for more robust and persistent tracking.
DEEPSORT_CONFIG = {
    'max_age': 50,          # The most important parameter. Increase this to keep tracks alive longer when they are not detected.
    'n_init': 5,            # Number of consecutive detections to confirm a track. Helps prevent false positives.
    'nms_max_overlap': 1.0, # NMS threshold for DeepSORT.
    'nn_budget': 100        # Size of the appearance gallery. A larger budget helps with long-term re-identification.
}

# PERFORMANCE: Batch size for model inference.
BATCH_SIZE = 4

# Analysis Parameters
LANE_LINE_DEFINITIONS = [
    [(100, 720), (500, 450)],  # Lane 1 Left
    [(550, 720), (650, 450)],  # Lane 1 Right / Lane 2 Left
    [(800, 720), (850, 450)],  # Lane 2 Right / Lane 3 Left
    [(1200, 720), (1100, 450)] # Lane 3 Right
]
CONGESTION_THRESHOLD = 10


# --- INITIALIZATION ---
print("--- Initializing Model and Tracker ---")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

model = YOLO(BEST_MODEL_PATH)
model.to(device)

# Initialize DeepSort tracker with our new stable configuration
tracker = DeepSort(
    max_age=DEEPSORT_CONFIG['max_age'],
    n_init=DEEPSORT_CONFIG['n_init'],
    nms_max_overlap=DEEPSORT_CONFIG['nms_max_overlap'],
    nn_budget=DEEPSORT_CONFIG['nn_budget']
)

# Video I/O
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened(): raise IOError(f"Error opening video: {VIDEO_PATH}")
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

# Lane Polygons and Data Structures
lane_polygons = [Polygon(LANE_LINE_DEFINITIONS[i] + LANE_LINE_DEFINITIONS[i+1][::-1]) for i in range(len(LANE_LINE_DEFINITIONS) - 1)]
total_vehicle_counts_by_lane = {i+1: defaultdict(set) for i in range(len(lane_polygons))}


# --- MAIN PROCESSING LOOP ---
print(f"--- Starting Stable Video Analysis ({total_frames} frames) ---")
frames_batch = []
frame_num = 0
start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    frames_batch.append(frame)
    frame_num += 1

    if len(frames_batch) == BATCH_SIZE or frame_num == total_frames:
        results_batch = model.predict(frames_batch, classes=TARGET_CLASSES, conf=CONF_THRESHOLD, iou=IOU_THRESHOLD, verbose=False)

        for i, results in enumerate(results_batch):
            current_frame = frames_batch[i]
            
            detections = []
            for box in results.boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                w_box, h_box = x2 - x1, y2 - y1
                detections.append(([int(x1), int(y1), int(w_box), int(h_box)], box.conf[0].item(), CLASS_NAMES[int(box.cls[0].item())]))
            
            tracks = tracker.update_tracks(detections, frame=current_frame)
            current_lane_counts = {i+1: 0 for i in range(len(lane_polygons))}
            
            for track in tracks:
                # We only process CONFIRMED tracks that have been recently updated.
                if not track.is_confirmed() or track.time_since_update > 1:
                    continue
                
                track_id, bbox, class_name = track.track_id, track.to_ltrb(), track.get_det_class()
                bottom_center_point = Point((bbox[0] + bbox[2]) / 2, bbox[3])
                assigned_lane = next((i + 1 for i, poly in enumerate(lane_polygons) if poly.contains(bottom_center_point)), None)
                
                if assigned_lane:
                    current_lane_counts[assigned_lane] += 1
                    total_vehicle_counts_by_lane[assigned_lane][class_name].add(track_id)
                
                label = f"{class_name} {track_id}" + (f" L:{assigned_lane}" if assigned_lane else "")
                cv2.rectangle(current_frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 255, 0), 2)
                cv2.putText(current_frame, label, (int(bbox[0]), int(bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

            y_offset = 40
            for lane_num, poly in enumerate(lane_polygons, 1):
                is_congested = current_lane_counts[lane_num] > CONGESTION_THRESHOLD
                lane_color = (0, 0, 255) if is_congested else (0, 255, 0)
                poly_pts = np.array(poly.exterior.coords, np.int32).reshape((-1, 1, 2))
                cv2.polylines(current_frame, [poly_pts], isClosed=True, color=lane_color, thickness=2)
                status = "Congested" if is_congested else "Normal"
                info_text = f"Lane {lane_num} ({status}): {current_lane_counts[lane_num]} vehicles"
                cv2.putText(current_frame, info_text, (20, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                y_offset += 40

            out.write(current_frame)

        frames_batch = []
        elapsed_time = time.time() - start_time
        processed_frames = frame_num - len(frames_batch)
        fps_current = processed_frames / elapsed_time if elapsed_time > 0 else 0
        print(f"Processed frame {processed_frames}/{total_frames}... Current FPS: {fps_current:.2f}")


# --- CLEANUP AND REPORT ---
cap.release()
out.release()
print(f"\n--- Analysis Complete ---")
print(f"Output video saved to: {OUTPUT_VIDEO_PATH}")

print("\n--- Final Vehicle Count Report (Unique Vehicles per Lane) ---")
for lane_num, class_counts in total_vehicle_counts_by_lane.items():
    total_in_lane = sum(len(ids) for ids in class_counts.values())
    print(f"\nLane {lane_num} | Total Unique Vehicles: {total_in_lane}")
    breakdown = ", ".join([f"{name}: {len(ids)}" for name, ids in class_counts.items() if len(ids) > 0])
    print(f"  > Breakdown: {breakdown}")

In [ ]:
# --- IMPORTS ---
import torch
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from shapely.geometry import Point, Polygon
import time
from collections import defaultdict, deque
import pandas as pd

# --- CONFIGURATION ---
print("\n--- Initializing Configuration ---")

# --- CORE PARAMETERS ---
MODEL_PATH = '/kaggle/working/runs/train/yolov8l_dropout_run/weights/best.pt'
VIDEO_PATH = '/kaggle/input/videohai/traffic.mp4'
OUTPUT_VIDEO_PATH = '/kaggle/working/traffic_intelligence_output.mp4'
CSV_LOG_PATH = '/kaggle/working/traffic_events_log.csv'

# --- DETECTION & TRACKING ---
TARGET_CLASSES = [0, 1, 2, 3] # 'car', 'truck', 'bus', 'motorcycle'
CLASS_NAMES = ['car', 'truck', 'bus', 'motorcycle']
CONF_THRESHOLD = 0.3
IOU_THRESHOLD = 0.5
DEEPSORT_CONFIG = {'max_age': 50, 'n_init': 5, 'nms_max_overlap': 1.0, 'nn_budget': 100}
BATCH_SIZE = 4 # Adjust based on VRAM

# --- ANALYSIS & VISUALIZATION ---
# Lane definitions
LANE_LINE_DEFINITIONS = [
    [(100, 720), (500, 450)],  # Lane 1 Left
    [(550, 720), (650, 450)],  # Lane 1 Right / Lane 2 Left
    [(800, 720), (850, 450)],  # Lane 2 Right / Lane 3 Left
    [(1200, 720), (1100, 450)] # Lane 3 Right
]
# **IMPORTANT**: Define the real-world length of the lanes in METERS for speed estimation.
# Measure the approximate vertical distance covered by the lanes in your video frame.
REAL_WORLD_LANE_LENGTH_METERS = 30 
# Define the expected flow of traffic ('up' or 'down' relative to the Y-axis)
LANE_FLOW_DIRECTION = 'up' # Vehicles move from high Y to low Y
# Thresholds for advanced analysis
CONGESTION_THRESHOLD = 10  # Vehicles per lane
STOPPED_VEHICLE_SPEED_THRESHOLD_KMPH = 3 # Speed below which a vehicle is considered stopped
STOPPED_VEHICLE_DURATION_SECONDS = 5 # How long a vehicle must be stopped to be flagged
SPEED_UNIT = 'km/h' # or 'mph'

# --- INITIALIZATION ---
print("--- Initializing Model, Tracker, and Video I/O ---")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = YOLO(MODEL_PATH)
model.to(device)
tracker = DeepSort(max_age=DEEPSORT_CONFIG['max_age'], n_init=DEEPSORT_CONFIG['n_init'], nms_max_overlap=DEEPSORT_CONFIG['nms_max_overlap'], nn_budget=DEEPSORT_CONFIG['nn_budget'])
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened(): raise IOError(f"Error opening video: {VIDEO_PATH}")
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

# --- DATA STRUCTURES ---
lane_polygons = [Polygon(LANE_LINE_DEFINITIONS[i] + LANE_LINE_DEFINITIONS[i+1][::-1]) for i in range(len(LANE_LINE_DEFINITIONS) - 1)]
track_history = defaultdict(lambda: deque(maxlen=int(fps))) # Store position history for speed calculation
stopped_timers = defaultdict(int)
event_log = []

# --- MAIN PROCESSING LOOP ---
print(f"--- Starting Advanced Video Analysis ({total_frames} frames) ---")
frames_batch = []
frame_num = 0
start_time = time.time()

try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break
        frames_batch.append(frame)
        frame_num += 1

        if len(frames_batch) == BATCH_SIZE or frame_num == total_frames:
            results_batch = model.predict(frames_batch, classes=TARGET_CLASSES, conf=CONF_THRESHOLD, iou=IOU_THRESHOLD, verbose=False)

            for i, results in enumerate(results_batch):
                current_frame_index = frame_num - len(frames_batch) + i
                current_frame_time = current_frame_index / fps
                current_frame = frames_batch[i]
                
                detections = []
                for box in results.boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    detections.append(([int(x1), int(y1), int(x2 - x1), int(y2 - y1)], box.conf[0].item(), CLASS_NAMES[int(box.cls[0].item())]))
                
                tracks = tracker.update_tracks(detections, frame=current_frame)
                current_lane_counts = {lane: 0 for lane in range(1, len(lane_polygons) + 1)}
                
                dashboard = np.zeros_like(current_frame, dtype=np.uint8)

                for track in tracks:
                    if not track.is_confirmed() or track.time_since_update > 1: continue
                    
                    track_id, bbox, class_name = track.track_id, track.to_ltrb(), track.get_det_class()
                    bottom_center = Point((bbox[0] + bbox[2]) / 2, bbox[3])
                    
                    # Update history and calculate speed
                    track_history[track_id].append(bottom_center)
                    speed_kmph = -1
                    if len(track_history[track_id]) > int(fps / 2): # Need at least half a second of history
                        prev_pos = track_history[track_id][0]
                        pixel_dist = bottom_center.distance(prev_pos)
                        time_diff = len(track_history[track_id]) / fps
                        
                        # Perspective correction for speed
                        perspective_scale = 1 - (bottom_center.y / h) # Closer objects (higher y) move more pixels
                        real_world_dist = pixel_dist * perspective_scale * (REAL_WORLD_LANE_LENGTH_METERS / h)
                        speed_mps = real_world_dist / time_diff
                        speed_kmph = speed_mps * (3.6 if SPEED_UNIT == 'km/h' else 2.23694)
                    
                    assigned_lane = next((i + 1 for i, poly in enumerate(lane_polygons) if poly.contains(bottom_center)), None)
                    if assigned_lane: current_lane_counts[assigned_lane] += 1

                    # --- ADVANCED ANALYSIS CHECKS ---
                    is_stopped = False
                    is_wrong_way = False

                    # Stopped vehicle check
                    if speed_kmph != -1 and speed_kmph < STOPPED_VEHICLE_SPEED_THRESHOLD_KMPH:
                        stopped_timers[track_id] += 1
                        if (stopped_timers[track_id] / fps) > STOPPED_VEHICLE_DURATION_SECONDS:
                            is_stopped = True
                            event_log.append({'timestamp': current_frame_time, 'event': 'stopped_vehicle', 'track_id': track_id, 'lane': assigned_lane, 'speed': speed_kmph})
                    else:
                        stopped_timers[track_id] = 0

                    # Wrong-way check
                    if len(track_history[track_id]) > 2:
                        direction_y = bottom_center.y - track_history[track_id][-2].y
                        flow = 'up' if direction_y < 0 else 'down'
                        if assigned_lane and flow != LANE_FLOW_DIRECTION:
                            is_wrong_way = True
                            event_log.append({'timestamp': current_frame_time, 'event': 'wrong_way', 'track_id': track_id, 'lane': assigned_lane})

                    # --- VISUALIZATION ---
                    label = f"{class_name} {track_id}"
                    color = (0, 255, 0) # Default green
                    if is_stopped:
                        label += " [STOPPED]"
                        color = (0, 0, 255) # Red for stopped
                    elif is_wrong_way:
                        label += " [WRONG WAY]"
                        color = (0, 255, 255) # Yellow for wrong way
                    elif speed_kmph > 0:
                        label += f" {int(speed_kmph)} {SPEED_UNIT}"

                    cv2.rectangle(current_frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), color, 2)
                    cv2.putText(current_frame, label, (int(bbox[0]), int(bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                
                # Draw Dashboard and Lane Polygons
                y_offset = 40
                for lane_num, poly in enumerate(lane_polygons, 1):
                    is_congested = current_lane_counts[lane_num] > CONGESTION_THRESHOLD
                    lane_color = (0, 0, 255) if is_congested else (0, 255, 0)
                    poly_pts = np.array(poly.exterior.coords, np.int32).reshape((-1, 1, 2))
                    cv2.polylines(current_frame, [poly_pts], isClosed=True, color=lane_color, thickness=2)
                    status = "Congested" if is_congested else "Normal"
                    info_text = f"Lane {lane_num} ({status}): {current_lane_counts[lane_num]} veh"
                    cv2.putText(dashboard, info_text, (20, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
                    y_offset += 30
                    if is_congested and not any(e['event'] == 'congestion' and e['lane'] == lane_num and abs(e['timestamp'] - current_frame_time) < 1 for e in event_log):
                        event_log.append({'timestamp': current_frame_time, 'event': 'congestion', 'lane': lane_num, 'vehicle_count': current_lane_counts[lane_num]})

                # Combine frame with dashboard
                final_frame = cv2.addWeighted(current_frame, 1, dashboard, 0.8, 0)
                out.write(final_frame)

            frames_batch = []
            elapsed_time = time.time() - start_time
            processed_frames = frame_num - len(frames_batch)
            fps_current = processed_frames / elapsed_time if elapsed_time > 0 else 0
            print(f"Processed frame {processed_frames}/{total_frames}... Current FPS: {fps_current:.2f}")

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # --- CLEANUP AND REPORT ---
    cap.release()
    out.release()
    print(f"\n--- Analysis Complete ---")
    print(f"Output video saved to: {OUTPUT_VIDEO_PATH}")

    # --- DATA EXPORT TO CSV ---
    if event_log:
        df = pd.DataFrame(event_log)
        df.to_csv(CSV_LOG_PATH, index=False)
        print(f"Event log saved to: {CSV_LOG_PATH}")
    else:
        print("No critical events were logged.")

In [48]:
# --- IMPORTS ---
import torch
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from shapely.geometry import Point, Polygon, LineString
import time
from collections import defaultdict, deque
import pandas as pd

# --- CONFIGURATION ---
print("\n--- Initializing Configuration ---")

# --- CORE PARAMETERS ---
MODEL_PATH = '/kaggle/working/runs/train/yolov8l_dropout_run/weights/best.pt'
VIDEO_PATH = '/kaggle/input/videohai/traffic.mp4'
OUTPUT_VIDEO_PATH = '/kaggle/working/highway_intelligence_final.mp4'
CSV_LOG_PATH = '/kaggle/working/highway_events_log_final.csv'

# --- DETECTION & TRACKING ---
TARGET_CLASSES = [0, 1, 2, 3]
CLASS_NAMES = ['car', 'truck', 'bus', 'motorcycle']
CONF_THRESHOLD = 0.3
IOU_THRESHOLD = 0.5
DEEPSORT_CONFIG = {'max_age': 50, 'n_init': 5, 'nms_max_overlap': 1.0, 'nn_budget': 100}
BATCH_SIZE = 4

# --- ANALYSIS & VISUALIZATION ---
LANE_LINE_DEFINITIONS = [
    [(100, 720), (500, 450)],
    [(550, 720), (650, 450)],
    [(800, 720), (850, 450)],
    [(1200, 720), (1100, 450)]
]
COUNTING_LINE_Y_POSITION = 650
DENSITY_THRESHOLDS = {'normal': 0.15, 'medium': 0.30}

# --- HIGHWAY SPEED ANALYSIS CONFIGURATION ---
REAL_WORLD_LANE_LENGTH_METERS = 30 
SPEED_CALIBRATION_FACTOR = 3.5 
SPEED_LIMIT_KMPH = 100
SPEED_UNIT = 'km/h'
STOPPED_VEHICLE_SPEED_THRESHOLD_KMPH = 5
STOPPED_VEHICLE_DURATION_SECONDS = 2
DASHBOARD_TRANSPARENCY = 0.6

# --- INITIALIZATION ---
print("--- Initializing Model, Tracker, and Video I/O ---")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = YOLO(MODEL_PATH)
model.to(device)
tracker = DeepSort(max_age=DEEPSORT_CONFIG['max_age'], n_init=DEEPSORT_CONFIG['n_init'], nms_max_overlap=DEEPSORT_CONFIG['nms_max_overlap'], nn_budget=DEEPSORT_CONFIG['nn_budget'])
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened(): raise IOError(f"Error opening video: {VIDEO_PATH}")
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

# --- DATA STRUCTURES ---
lane_polygons = [Polygon(LANE_LINE_DEFINITIONS[i] + LANE_LINE_DEFINITIONS[i+1][::-1]) for i in range(len(LANE_LINE_DEFINITIONS) - 1)]
lane_areas_pixels = [poly.area for poly in lane_polygons]
track_history = defaultdict(lambda: deque(maxlen=int(fps)))
stopped_timers = defaultdict(int)
cumulative_counts_by_lane = defaultdict(int)
counted_ids_by_lane = defaultdict(set)
counting_line = LineString([(0, COUNTING_LINE_Y_POSITION), (w, COUNTING_LINE_Y_POSITION)])
event_log = []

# --- MAIN PROCESSING LOOP ---
print(f"--- Starting Highway Analysis ({total_frames} frames) ---")
frames_batch = []
frame_num = 0
start_time = time.time()

try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break
        frames_batch.append(frame)
        frame_num += 1

        if len(frames_batch) == BATCH_SIZE or frame_num == total_frames:
            results_batch = model.predict(frames_batch, classes=TARGET_CLASSES, conf=CONF_THRESHOLD, iou=IOU_THRESHOLD, verbose=False)

            for i, results in enumerate(results_batch):
                current_frame = frames_batch[i]
                
                detections = []
                for box in results.boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    detections.append(([int(x1), int(y1), int(x2 - x1), int(y2 - y1)], box.conf[0].item(), CLASS_NAMES[int(box.cls[0].item())]))
                
                tracks = tracker.update_tracks(detections, frame=current_frame)
                current_lane_counts = defaultdict(int)
                lane_vehicle_areas = defaultdict(float)

                for track in tracks:
                    if not track.is_confirmed() or track.time_since_update > 1: continue
                    
                    track_id, bbox, class_name = track.track_id, track.to_ltrb(), track.get_det_class()
                    
                    # --- BUG FIX: Initialize speed_kmph to a default value for every track ---
                    speed_kmph = -1
                    
                    bottom_center = Point((bbox[0] + bbox[2]) / 2, bbox[3])
                    track_history[track_id].append(bottom_center)
                    
                    if len(track_history[track_id]) > int(fps / 2):
                        prev_pos = track_history[track_id][0]
                        pixel_dist = bottom_center.distance(prev_pos)
                        time_diff = len(track_history[track_id]) / fps
                        perspective_scale = 1 - (bottom_center.y / h)
                        real_world_dist = pixel_dist * perspective_scale * (REAL_WORLD_LANE_LENGTH_METERS / h)
                        speed_mps = real_world_dist / time_diff if time_diff > 0 else 0
                        speed_kmph = speed_mps * 3.6 * SPEED_CALIBRATION_FACTOR

                    is_stopped = speed_kmph < STOPPED_VEHICLE_SPEED_THRESHOLD_KMPH and speed_kmph != -1
                    is_speeding = speed_kmph > SPEED_LIMIT_KMPH

                    # ... (Rest of the analysis and visualization logic) ...
                    assigned_lane = next((i + 1 for i, poly in enumerate(lane_polygons) if poly.contains(bottom_center)), None)
                    if assigned_lane:
                        current_lane_counts[assigned_lane] += 1
                        bbox_area = (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
                        lane_vehicle_areas[assigned_lane] += bbox_area
                    if len(track_history[track_id]) > 1 and assigned_lane is not None:
                        if LineString([track_history[track_id][-2], track_history[track_id][-1]]).intersects(counting_line):
                            if track_id not in counted_ids_by_lane[assigned_lane]:
                                cumulative_counts_by_lane[assigned_lane] += 1
                                counted_ids_by_lane[assigned_lane].add(track_id)

                    if is_speeding: label, color = f"{class_name} {track_id} [{int(speed_kmph)} {SPEED_UNIT}] SPEEDING", (255, 0, 255)
                    elif is_stopped:
                        stopped_timers[track_id] += 1
                        if (stopped_timers[track_id] / fps) > STOPPED_VEHICLE_DURATION_SECONDS: label, color = f"{class_name} {track_id} [STOPPED]", (0, 0, 255)
                        else: label, color = f"{class_name} {track_id} {int(speed_kmph)} {SPEED_UNIT}", (0, 255, 0)
                    else:
                        stopped_timers[track_id] = 0
                        label = f"{class_name} {track_id} {int(speed_kmph)} {SPEED_UNIT}" if speed_kmph > 0 else f"{class_name} {track_id}"
                        color = (0, 255, 0)
                    cv2.rectangle(current_frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), color, 2)
                    cv2.putText(current_frame, label, (int(bbox[0]), int(bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

                # --- Dashboard drawing logic remains the same ---
                dashboard_panel = np.zeros((200, w, 3), dtype=np.uint8)
                y_offset = 30
                for lane_num, poly in enumerate(lane_polygons, 1):
                    density_ratio = lane_vehicle_areas[lane_num] / lane_areas_pixels[lane_num - 1] if lane_areas_pixels[lane_num - 1] > 0 else 0
                    if density_ratio <= DENSITY_THRESHOLDS['normal']: density_level = "Normal"
                    elif density_ratio <= DENSITY_THRESHOLDS['medium']: density_level = "Medium"
                    else: density_level = "High"
                    lane_color = (0, 165, 255) if density_level == "Medium" else (0, 0, 255) if density_level == "High" else (0, 255, 0)
                    poly_pts = np.array(poly.exterior.coords, np.int32).reshape((-1, 1, 2))
                    cv2.polylines(current_frame, [poly_pts], isClosed=True, color=lane_color, thickness=2)
                    info_text = f"Lane {lane_num} | Density: {density_level} ({density_ratio:.1%}) | Live Count: {current_lane_counts[lane_num]}"
                    cv2.putText(dashboard_panel, info_text, (20, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                    y_offset += 30
                cv2.putText(dashboard_panel, "Cumulative Vehicle Count (Total Passed)", (20, y_offset + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
                y_offset += 50
                count_text = "  ".join([f"Lane {ln}: {cnt}" for ln, cnt in sorted(cumulative_counts_by_lane.items())])
                cv2.putText(dashboard_panel, count_text, (20, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                roi = current_frame[0:200, 0:w]
                blended_roi = cv2.addWeighted(roi, 1.0, dashboard_panel, DASHBOARD_TRANSPARENCY, 0)
                current_frame[0:200, 0:w] = blended_roi
                cv2.line(current_frame, (0, COUNTING_LINE_Y_POSITION), (w, COUNTING_LINE_Y_POSITION), (255, 255, 0), 2)
                out.write(current_frame)

            frames_batch = []
            elapsed_time = time.time() - start_time
            processed_frames = frame_num - len(frames_batch)
            fps_current = processed_frames / elapsed_time if elapsed_time > 0 else 0
            print(f"Processed frame {processed_frames}/{total_frames}... Current FPS: {fps_current:.2f}")

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    cap.release()
    out.release()
    print(f"\n--- Analysis Complete ---")
    print(f"Output video saved to: {OUTPUT_VIDEO_PATH}")
    if event_log:
        df = pd.DataFrame(event_log)
        df.to_csv(CSV_LOG_PATH, index=False)
        print(f"Event log saved to: {CSV_LOG_PATH}")


--- Initializing Configuration ---
--- Initializing Model, Tracker, and Video I/O ---
--- Starting Highway Analysis (900 frames) ---
Processed frame 4/900... Current FPS: 3.63
Processed frame 8/900... Current FPS: 5.79
Processed frame 12/900... Current FPS: 7.19
Processed frame 16/900... Current FPS: 8.12
Processed frame 20/900... Current FPS: 8.82
Processed frame 24/900... Current FPS: 9.36
Processed frame 28/900... Current FPS: 9.76
Processed frame 32/900... Current FPS: 10.08
Processed frame 36/900... Current FPS: 10.32
Processed frame 40/900... Current FPS: 10.52
Processed frame 44/900... Current FPS: 10.69
Processed frame 48/900... Current FPS: 10.86
Processed frame 52/900... Current FPS: 10.98
Processed frame 56/900... Current FPS: 11.07
Processed frame 60/900... Current FPS: 11.15
Processed frame 64/900... Current FPS: 11.31
Processed frame 68/900... Current FPS: 11.38
Processed frame 72/900... Current FPS: 11.45
Processed frame 76/900... Current FPS: 11.51
Processed frame 80/9

In [51]:
# --- IMPORTS ---
import torch
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from shapely.geometry import Point, Polygon, LineString
import time
from collections import defaultdict, deque
import pandas as pd

# --- CONFIGURATION ---
print("\n--- Initializing Configuration ---")

# --- CORE PARAMETERS ---
MODEL_PATH = '/kaggle/working/runs/train/yolov8l_dropout_run/weights/best.pt'
VIDEO_PATH = '/kaggle/input/videohai/traffic.mp4'
OUTPUT_VIDEO_PATH = '/kaggle/working/traffic_platform_final.mp4'
CSV_LOG_PATH = '/kaggle/working/traffic_platform_events_final.csv'

# --- DETECTION & TRACKING ---
TARGET_CLASSES = [0, 1, 2, 3]
CLASS_NAMES = ['car', 'truck', 'bus', 'motorcycle']
CONF_THRESHOLD = 0.3
IOU_THRESHOLD = 0.5
DEEPSORT_CONFIG = {'max_age': 50, 'n_init': 5, 'nms_max_overlap': 1.0, 'nn_budget': 100}
BATCH_SIZE = 4

# --- ANALYSIS & VISUALIZATION ---
LANE_LINE_DEFINITIONS = [
    [(100, 720), (500, 450)], [(550, 720), (650, 450)],
    [(800, 720), (850, 450)], [(1200, 720), (1100, 450)]
]
COUNTING_LINE_Y_POSITION = 400
DENSITY_THRESHOLDS = {'normal': 0.15, 'medium': 0.30}
SPEED_CALIBRATION_FACTOR = 3.5 
SPEED_LIMIT_KMPH = 100
SPEED_UNIT = 'km/h'
STOPPED_VEHICLE_SPEED_THRESHOLD_KMPH = 5
STOPPED_VEHICLE_DURATION_SECONDS = 5
DASHBOARD_TRANSPARENCY = 0.7

# --- ADVANCED FEATURE CONFIGURATION ---
LANE_CHANGE_COOLDOWN_FRAMES = 15
SUDDEN_BRAKING_THRESHOLD_FACTOR = 0.5
HEATMAP_DECAY_RATE = 0.98

# --- INITIALIZATION ---
print("--- Initializing Systems ---")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = YOLO(MODEL_PATH)
model.to(device)
tracker = DeepSort(max_age=DEEPSORT_CONFIG['max_age'], n_init=DEEPSORT_CONFIG['n_init'], nms_max_overlap=DEEPSORT_CONFIG['nms_max_overlap'], nn_budget=DEEPSORT_CONFIG['nn_budget'])
cap = cv2.VideoCapture(VIDEO_PATH)
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

# --- DATA STRUCTURES ---
lane_polygons = [Polygon(LANE_LINE_DEFINITIONS[i] + LANE_LINE_DEFINITIONS[i+1][::-1]) for i in range(len(LANE_LINE_DEFINITIONS) - 1)]
lane_areas_pixels = [poly.area for poly in lane_polygons]
track_history = defaultdict(lambda: deque(maxlen=int(fps)))
stopped_timers, event_log = defaultdict(int), []
cumulative_counts_by_lane, counted_ids_by_lane = defaultdict(int), defaultdict(set)
counting_line = LineString([(0, COUNTING_LINE_Y_POSITION), (w, COUNTING_LINE_Y_POSITION)])
lane_speed_history = defaultdict(lambda: deque(maxlen=int(fps * 2)))
track_speed_history = defaultdict(lambda: deque(maxlen=5))
track_lane_history = defaultdict(int)
lane_change_timers = defaultdict(int)
heatmap_layer = np.zeros((h, w), dtype=np.float32)

# --- MAIN PROCESSING LOOP ---
print(f"--- Starting Advanced Analysis ---")
frames_batch, frame_num, start_time = [], 0, time.time()

try:
    while cap.isOpened():
        ret, frame = cap.read();
        if not ret: break
        frames_batch.append(frame); frame_num += 1

        if len(frames_batch) == BATCH_SIZE or frame_num == int(cap.get(cv2.CAP_PROP_FRAME_COUNT)):
            results_batch = model.predict(frames_batch, classes=TARGET_CLASSES, conf=CONF_THRESHOLD, iou=IOU_THRESHOLD, verbose=False)

            for i, results in enumerate(results_batch):
                current_frame = frames_batch[i]
                
                detections = []
                for box in results.boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    detections.append(([int(x1), int(y1), int(x2 - x1), int(y2 - y1)], box.conf[0].item(), CLASS_NAMES[int(box.cls[0].item())]))
                
                tracks = tracker.update_tracks(detections, frame=current_frame)
                current_lane_counts, lane_vehicle_areas = defaultdict(int), defaultdict(float)
                heatmap_layer = cv2.addWeighted(heatmap_layer, HEATMAP_DECAY_RATE, np.zeros_like(heatmap_layer), 1 - HEATMAP_DECAY_RATE, 0)
                all_moving_speeds = []

                for track in tracks:
                    # ... Main track processing ... (This part remains the same)
                    if not track.is_confirmed() or track.time_since_update > 1: continue
                    track_id, bbox, class_name = track.track_id, track.to_ltrb(), track.get_det_class()
                    bottom_center = Point((bbox[0] + bbox[2]) / 2, bbox[3])
                    track_history[track_id].append(bottom_center)
                    cv2.circle(heatmap_layer, (int(bottom_center.x), int(bottom_center.y)), 10, (255), -1)
                    # ... All the speed, behavior, and lane logic ...
                    speed_kmph, assigned_lane = -1, next((i + 1 for i, poly in enumerate(lane_polygons) if poly.contains(bottom_center)), None)
                    if len(track_history[track_id]) > int(fps / 2):
                        prev_pos, time_diff = track_history[track_id][0], len(track_history[track_id]) / fps
                        pixel_dist, perspective_scale = bottom_center.distance(prev_pos), 1 - (bottom_center.y / h)
                        real_world_dist = pixel_dist * perspective_scale * (REAL_WORLD_LANE_LENGTH_METERS / h)
                        speed_kmph = (real_world_dist / time_diff * 3.6 if time_diff > 0 else 0) * SPEED_CALIBRATION_FACTOR
                    if assigned_lane:
                        current_lane_counts[assigned_lane] += 1
                        lane_vehicle_areas[assigned_lane] += (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
                        if speed_kmph > STOPPED_VEHICLE_SPEED_THRESHOLD_KMPH:
                            lane_speed_history[assigned_lane].append(speed_kmph)
                            all_moving_speeds.append(speed_kmph)
                    is_stopped, is_speeding, is_braking, is_changing_lanes = False, False, False, False
                    is_speeding = speed_kmph > SPEED_LIMIT_KMPH
                    if speed_kmph != -1 and speed_kmph < STOPPED_VEHICLE_SPEED_THRESHOLD_KMPH:
                        stopped_timers[track_id] += 1
                        if (stopped_timers[track_id] / fps) > STOPPED_VEHICLE_DURATION_SECONDS: is_stopped = True
                    else: stopped_timers[track_id] = 0
                    if track_speed_history[track_id] and speed_kmph != -1:
                        if speed_kmph < track_speed_history[track_id][-1] * SUDDEN_BRAKING_THRESHOLD_FACTOR: is_braking = True
                    track_speed_history[track_id].append(speed_kmph)
                    if assigned_lane and track_lane_history[track_id] != 0 and assigned_lane != track_lane_history[track_id]:
                        is_changing_lanes = True; lane_change_timers[track_id] = LANE_CHANGE_COOLDOWN_FRAMES
                    if lane_change_timers[track_id] > 0:
                        is_changing_lanes = True; lane_change_timers[track_id] -= 1
                    track_lane_history[track_id] = assigned_lane
                    if is_speeding: label, color = f"{class_name} {track_id} [SPEEDING]", (255, 0, 255)
                    elif is_stopped: label, color = f"{class_name} {track_id} [STOPPED]", (0, 0, 255)
                    elif is_braking: label, color = f"{class_name} {track_id} [BRAKING]", (0, 165, 255)
                    elif is_changing_lanes: label, color = f"{class_name} {track_id} [LANE CHANGE]", (255, 255, 0)
                    else:
                        label = f"{class_name} {track_id} {int(speed_kmph)} {SPEED_UNIT}" if speed_kmph > 0 else f"{class_name} {track_id}"
                        color = (0, 255, 0)
                    cv2.rectangle(current_frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), color, 2)
                    cv2.putText(current_frame, label, (int(bbox[0]), int(bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                
                # --- VISUALIZATION ---
                # --- THIS IS THE FIX ---
                # Step 1: Normalize the heatmap to a float array in the range [0, 255]
                heatmap_normalized_float = cv2.normalize(heatmap_layer, None, 0, 255, cv2.NORM_MINMAX)
                # Step 2: Convert the float array to an 8-bit unsigned integer array
                heatmap_uint8 = heatmap_normalized_float.astype(np.uint8)
                # Step 3: Apply the colormap
                heatmap_colored = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET)

                final_frame = cv2.addWeighted(current_frame, 0.6, heatmap_colored, 0.4, 0)
                dashboard_panel = np.zeros((200, w, 3), dtype=np.uint8)
                y_offset = 30
                avg_scene_speed = int(np.mean(all_moving_speeds)) if all_moving_speeds else 0
                cv2.putText(dashboard_panel, f"Scene Analytics: {sum(current_lane_counts.values())} Vehicles Visible | Avg Speed: {avg_scene_speed} {SPEED_UNIT}", (20, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                y_offset += 40

                for lane_num, poly in enumerate(lane_polygons, 1):
                    # ... (Dashboard text logic remains the same) ...
                    avg_lane_speed = int(np.mean(lane_speed_history[lane_num])) if lane_speed_history[lane_num] else 0
                    trend = "→"
                    if len(lane_speed_history[lane_num]) > 10:
                        recent_avg = np.mean(list(lane_speed_history[lane_num])[-10:])
                        older_avg = np.mean(list(lane_speed_history[lane_num])[:-10])
                        if recent_avg > older_avg * 1.1: trend = "↑"
                        elif recent_avg < older_avg * 0.9: trend = "↓"
                    density_ratio = lane_vehicle_areas[lane_num] / lane_areas_pixels[lane_num - 1] if lane_areas_pixels[lane_num - 1] > 0 else 0
                    density_level = "High" if density_ratio > DENSITY_THRESHOLDS['medium'] else "Medium" if density_ratio > DENSITY_THRESHOLDS['normal'] else "Normal"
                    info_text = f"Lane {lane_num}: {density_level} ({density_ratio:.1%}) | Avg Speed: {avg_lane_speed} {SPEED_UNIT} {trend} | Live: {current_lane_counts[lane_num]}"
                    cv2.putText(dashboard_panel, info_text, (20, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                    y_offset += 30

                roi = final_frame[0:200, 0:w]
                blended_roi = cv2.addWeighted(roi, 1.0, dashboard_panel, DASHBOARD_TRANSPARENCY, 0)
                final_frame[0:200, 0:w] = blended_roi
                out.write(final_frame)

            frames_batch = [];
            print(f"Processed frame {frame_num}/{int(cap.get(cv2.CAP_PROP_FRAME_COUNT))}...")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    cap.release(); out.release(); print("\n--- Analysis Complete ---")


--- Initializing Configuration ---
--- Initializing Systems ---
--- Starting Advanced Analysis ---
Processed frame 4/900...
Processed frame 8/900...
Processed frame 12/900...
Processed frame 16/900...
Processed frame 20/900...
Processed frame 24/900...
Processed frame 28/900...
Processed frame 32/900...
Processed frame 36/900...
Processed frame 40/900...
Processed frame 44/900...
Processed frame 48/900...
Processed frame 52/900...
Processed frame 56/900...
Processed frame 60/900...
Processed frame 64/900...
Processed frame 68/900...
Processed frame 72/900...
Processed frame 76/900...
Processed frame 80/900...
Processed frame 84/900...
Processed frame 88/900...
Processed frame 92/900...
Processed frame 96/900...
Processed frame 100/900...
Processed frame 104/900...
Processed frame 108/900...
Processed frame 112/900...
Processed frame 116/900...
Processed frame 120/900...
Processed frame 124/900...
Processed frame 128/900...
Processed frame 132/900...
Processed frame 136/900...
Processe

In [53]:
# --- IMPORTS ---
import torch
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from shapely.geometry import Point, Polygon, LineString
import time
from collections import defaultdict, deque
import pandas as pd

# --- CONFIGURATION ---
print("\n--- Initializing Configuration ---")

# --- CORE PARAMETERS ---
MODEL_PATH = '/kaggle/working/runs/train/yolov8l_dropout_run/weights/best.pt'
VIDEO_PATH = '/kaggle/input/videohai/traffic.mp4'
OUTPUT_VIDEO_PATH = '/kaggle/working/traffic_LoS_analysis_final.mp4'
CSV_LOG_PATH = '/kaggle/working/traffic_LoS_events_final.csv'

# --- DETECTION & TRACKING ---
TARGET_CLASSES = [0, 1, 2, 3]
CLASS_NAMES = ['car', 'truck', 'bus', 'motorcycle']
CONF_THRESHOLD = 0.3
IOU_THRESHOLD = 0.5
DEEPSORT_CONFIG = {'max_age': 50, 'n_init': 5, 'nms_max_overlap': 1.0, 'nn_budget': 100}
BATCH_SIZE = 4

# --- ADVANCED TRAFFIC ANALYSIS CONFIGURATION ---
LANE_LINE_DEFINITIONS = [
    [(100, 720), (500, 450)], [(550, 720), (650, 450)],
    [(800, 720), (850, 450)], [(1200, 720), (1100, 450)]
]
COUNTING_LINE_Y_POSITION = 400
DASHBOARD_TRANSPARENCY = 0.7
SPEED_CALIBRATION_FACTOR = 3.5 
REAL_WORLD_LANE_LENGTH_METERS = 30 
SPEED_UNIT = 'km/h'
LOS_THRESHOLDS = {'A': 0.10, 'B': 0.18, 'C': 0.26, 'D': 0.35, 'E': 0.45}
LOS_COLORS = {'A': (0, 255, 0), 'B': (100, 255, 0), 'C': (200, 255, 0), 'D': (255, 200, 0), 'E': (255, 100, 0), 'F': (255, 0, 0)}
FLOW_RATE_WINDOW_SECONDS = 60
QUEUE_DETECTION_SPEED_KMPH = 10
QUEUE_MIN_VEHICLES = 3

# --- INITIALIZATION ---
print("--- Initializing Systems ---")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = YOLO(MODEL_PATH)
model.to(device)
tracker = DeepSort(max_age=DEEPSORT_CONFIG['max_age'], n_init=DEEPSORT_CONFIG['n_init'], nms_max_overlap=DEEPSORT_CONFIG['nms_max_overlap'], nn_budget=DEEPSORT_CONFIG['nn_budget'])
cap = cv2.VideoCapture(VIDEO_PATH)
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

# --- DATA STRUCTURES ---
lane_polygons = [Polygon(LANE_LINE_DEFINITIONS[i] + LANE_LINE_DEFINITIONS[i+1][::-1]) for i in range(len(LANE_LINE_DEFINITIONS) - 1)]
lane_areas_pixels = [poly.area for poly in lane_polygons]
track_history = defaultdict(lambda: deque(maxlen=int(fps)))
event_log = []
counting_line = LineString([(0, COUNTING_LINE_Y_POSITION), (w, COUNTING_LINE_Y_POSITION)])
lane_speed_history = defaultdict(lambda: deque(maxlen=int(fps * 2)))
flow_rate_timestamps = defaultdict(lambda: deque())
track_statuses = {}

# --- MAIN PROCESSING LOOP ---
print(f"--- Starting Level of Service (LoS) Analysis ---")
frames_batch, frame_num, start_time = [], 0, time.time()

try:
    while cap.isOpened():
        ret, frame = cap.read();
        if not ret: break
        frames_batch.append(frame); frame_num += 1

        if len(frames_batch) == BATCH_SIZE or frame_num == int(cap.get(cv2.CAP_PROP_FRAME_COUNT)):
            results_batch = model.predict(frames_batch, classes=TARGET_CLASSES, conf=CONF_THRESHOLD, iou=IOU_THRESHOLD, verbose=False)

            for i, results in enumerate(results_batch):
                current_frame = frames_batch[i]
                current_frame_time = (frame_num - len(frames_batch) + i) / fps
                
                # --- THIS IS THE FIX ---
                # We must manually build the detections list in the correct format for DeepSORT
                detections = []
                for box in results.boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    w_box, h_box = x2 - x1, y2 - y1
                    conf = box.conf[0].item()
                    cls = CLASS_NAMES[int(box.cls[0].item())]
                    detections.append(([int(x1), int(y1), int(w_box), int(h_box)], conf, cls))
                
                # Now pass the correctly formatted list to the tracker
                tracks = tracker.update_tracks(detections, frame=current_frame)
                
                current_lane_counts, lane_vehicle_areas = defaultdict(int), defaultdict(float)
                track_statuses.clear()

                for track in tracks:
                    # ... (The rest of the logic remains the same) ...
                    if not track.is_confirmed() or track.time_since_update > 1: continue
                    track_id, bbox, class_name = track.track_id, track.to_ltrb(), track.get_det_class()
                    bottom_center = Point((bbox[0] + bbox[2]) / 2, bbox[3])
                    track_history[track_id].append(bottom_center)
                    speed_kmph, assigned_lane = -1, next((i + 1 for i, poly in enumerate(lane_polygons) if poly.contains(bottom_center)), None)
                    if len(track_history[track_id]) > int(fps / 2):
                        prev_pos, time_diff = track_history[track_id][0], len(track_history[track_id]) / fps
                        pixel_dist, perspective_scale = bottom_center.distance(prev_pos), 1 - (bottom_center.y / h)
                        real_world_dist = pixel_dist * perspective_scale * (REAL_WORLD_LANE_LENGTH_METERS / h)
                        speed_kmph = (real_world_dist / time_diff * 3.6 if time_diff > 0 else 0) * SPEED_CALIBRATION_FACTOR
                    track_statuses[track_id] = {'speed': speed_kmph, 'bbox': bbox, 'lane': assigned_lane}
                    if assigned_lane:
                        current_lane_counts[assigned_lane] += 1
                        lane_vehicle_areas[assigned_lane] += (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
                        if speed_kmph > 0: lane_speed_history[assigned_lane].append(speed_kmph)
                    if len(track_history[track_id]) > 1:
                        if LineString([track_history[track_id][-2], track_history[track_id][-1]]).intersects(counting_line):
                            flow_rate_timestamps[assigned_lane].append(current_frame_time)
                    label = f"{class_name} {track_id} {int(speed_kmph)} {SPEED_UNIT}" if speed_kmph > 0 else f"{class_name} {track_id}"
                    cv2.rectangle(current_frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 255, 0), 2)
                    cv2.putText(current_frame, label, (int(bbox[0]), int(bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                dashboard_panel = np.zeros((220, w, 3), dtype=np.uint8)
                y_offset = 30
                for lane_num in range(1, len(lane_polygons) + 1):
                    queued_vehicles = [t for t_id, t in track_statuses.items() if t['lane'] == lane_num and t['speed'] != -1 and t['speed'] < QUEUE_DETECTION_SPEED_KMPH]
                    if len(queued_vehicles) >= QUEUE_MIN_VEHICLES:
                        min_y, max_y = min(v['bbox'][1] for v in queued_vehicles), max(v['bbox'][3] for v in queued_vehicles)
                        queue_pixel_length, avg_y = max_y - min_y, (min_y + max_y) / 2
                        perspective_scale = 1 - (avg_y / h)
                        queue_meters = queue_pixel_length * perspective_scale * (REAL_WORLD_LANE_LENGTH_METERS / h)
                        poly_pts = np.array(lane_polygons[lane_num-1].exterior.coords, np.int32)
                        overlay = current_frame.copy()
                        cv2.fillPoly(overlay, [poly_pts], (0, 0, 150))
                        current_frame = cv2.addWeighted(overlay, 0.4, current_frame, 0.6, 0)
                        queue_x_pos = int(lane_polygons[lane_num-1].centroid.x)
                        cv2.line(current_frame, (queue_x_pos + 20, int(min_y)), (queue_x_pos + 20, int(max_y)), (0, 255, 255), 3)
                        cv2.putText(current_frame, f"{queue_meters:.1f}m Queue", (queue_x_pos + 25, int(avg_y)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

                for lane_num, poly in enumerate(lane_polygons, 1):
                    density_ratio = lane_vehicle_areas[lane_num] / lane_areas_pixels[lane_num - 1] if lane_areas_pixels[lane_num - 1] > 0 else 0
                    los = 'F';
                    for grade, threshold in sorted(LOS_THRESHOLDS.items()):
                        if density_ratio <= threshold: los = grade; break
                    while flow_rate_timestamps[lane_num] and flow_rate_timestamps[lane_num][0] < current_frame_time - FLOW_RATE_WINDOW_SECONDS:
                        flow_rate_timestamps[lane_num].popleft()
                    flow_rate_vph = len(flow_rate_timestamps[lane_num]) * (3600 / FLOW_RATE_WINDOW_SECONDS)
                    avg_lane_speed = int(np.mean(lane_speed_history[lane_num])) if lane_speed_history[lane_num] else 0
                    trend = "→"
                    if len(lane_speed_history[lane_num]) > fps:
                        recent_avg, older_avg = np.mean(list(lane_speed_history[lane_num])[-int(fps/2):]), np.mean(list(lane_speed_history[lane_num])[:-int(fps/2)])
                        if recent_avg > older_avg * 1.1: trend = "↑"
                        elif recent_avg < older_avg * 0.9: trend = "↓"
                    info_text = f"Lane {lane_num} | LoS: {los} | Flow: {int(flow_rate_vph)} VPH | Avg Speed: {avg_lane_speed} {trend} | Live: {current_lane_counts[lane_num]}"
                    cv2.putText(dashboard_panel, info_text, (20, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, LOS_COLORS[los], 2)
                    y_offset += 30

                roi = current_frame[0:220, 0:w]
                blended_roi = cv2.addWeighted(roi, 1.0, dashboard_panel, DASHBOARD_TRANSPARENCY, 0)
                current_frame[0:220, 0:w] = blended_roi
                out.write(current_frame)

            frames_batch = []; print(f"Processed frame {frame_num}/{int(cap.get(cv2.CAP_PROP_FRAME_COUNT))}...")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    cap.release(); out.release(); print("\n--- Analysis Complete ---")


--- Initializing Configuration ---
--- Initializing Systems ---
--- Starting Level of Service (LoS) Analysis ---
Processed frame 4/900...
Processed frame 8/900...
Processed frame 12/900...
Processed frame 16/900...
Processed frame 20/900...
Processed frame 24/900...
Processed frame 28/900...
Processed frame 32/900...
Processed frame 36/900...
Processed frame 40/900...
Processed frame 44/900...
Processed frame 48/900...
Processed frame 52/900...
Processed frame 56/900...
Processed frame 60/900...
Processed frame 64/900...
Processed frame 68/900...
Processed frame 72/900...
Processed frame 76/900...
Processed frame 80/900...
Processed frame 84/900...
Processed frame 88/900...
Processed frame 92/900...
Processed frame 96/900...
Processed frame 100/900...
Processed frame 104/900...
Processed frame 108/900...
Processed frame 112/900...
Processed frame 116/900...
Processed frame 120/900...
Processed frame 124/900...
Processed frame 128/900...
Processed frame 132/900...
Processed frame 136/9

In [ ]:
# --- IMPORTS ---
import torch
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from shapely.geometry import Point, Polygon
import time
from collections import defaultdict, deque
import pandas as pd

# --- CONFIGURATION ---
print("\n--- Initializing Configuration ---")

# --- CORE PARAMETERS ---
MODEL_PATH = '/kaggle/working/runs/train/yolov8l_dropout_run/weights/best.pt'
VIDEO_PATH = '/kaggle/input/videohai/traffic.mp4'
OUTPUT_VIDEO_PATH = '/kaggle/working/traffic_analysis_final.mp4'
CSV_LOG_PATH = '/kaggle/working/traffic_events_log_final.csv'

# --- DETECTION & TRACKING ---
TARGET_CLASSES = [0, 1, 2, 3] # 'car', 'truck', 'bus', 'motorcycle'
CLASS_NAMES = ['car', 'truck', 'bus', 'motorcycle']
CONF_THRESHOLD = 0.3
IOU_THRESHOLD = 0.5
DEEPSORT_CONFIG = {'max_age': 50, 'n_init': 5, 'nms_max_overlap': 1.0, 'nn_budget': 100}
BATCH_SIZE = 4

# --- ANALYSIS & VISUALIZATION ---
# Lane definitions
LANE_LINE_DEFINITIONS = [
    [(100, 720), (500, 450)],  # Lane 1 Left
    [(550, 720), (650, 450)],  # Lane 1 Right / Lane 2 Left
    [(800, 720), (850, 450)],  # Lane 2 Right / Lane 3 Left
    [(1200, 720), (1100, 450)] # Lane 3 Right
]
# **IMPORTANT**: Define the real-world length of the lanes in METERS for speed estimation.
REAL_WORLD_LANE_LENGTH_METERS = 20
SPEED_UNIT = 'km/h' # or 'mph'

# Thresholds for stopped vehicle and congestion analysis
CONGESTION_THRESHOLD = 5  # Recommended: 6-10 for urban roads, 10-15 for highways
STOPPED_VEHICLE_SPEED_THRESHOLD_KMPH = 1.5  # 0–2 km/h is effectively stopped
STOPPED_VEHICLE_DURATION_SECONDS = 1.5  # Lower this for quicker response; 3–5 sec is good

# --- REMOVED --- The LANE_FLOW_DIRECTION parameter has been removed as it was causing incorrect detections.

# --- INITIALIZATION ---
print("--- Initializing Model, Tracker, and Video I/O ---")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = YOLO(MODEL_PATH)
model.to(device)
tracker = DeepSort(max_age=DEEPSORT_CONFIG['max_age'], n_init=DEEPSORT_CONFIG['n_init'], nms_max_overlap=DEEPSORT_CONFIG['nms_max_overlap'], nn_budget=DEEPSORT_CONFIG['nn_budget'])
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened(): raise IOError(f"Error opening video: {VIDEO_PATH}")
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

# --- DATA STRUCTURES ---
lane_polygons = [Polygon(LANE_LINE_DEFINITIONS[i] + LANE_LINE_DEFINITIONS[i+1][::-1]) for i in range(len(LANE_LINE_DEFINITIONS) - 1)]
track_history = defaultdict(lambda: deque(maxlen=int(fps)))
stopped_timers = defaultdict(int)
event_log = []

# --- MAIN PROCESSING LOOP ---
print(f"--- Starting Refined Video Analysis ({total_frames} frames) ---")
frames_batch = []
frame_num = 0
start_time = time.time()

try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break
        frames_batch.append(frame)
        frame_num += 1

        if len(frames_batch) == BATCH_SIZE or frame_num == total_frames:
            results_batch = model.predict(frames_batch, classes=TARGET_CLASSES, conf=CONF_THRESHOLD, iou=IOU_THRESHOLD, verbose=False)

            for i, results in enumerate(results_batch):
                current_frame_index = frame_num - len(frames_batch) + i
                current_frame_time = current_frame_index / fps
                current_frame = frames_batch[i]
                
                detections = []
                for box in results.boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    detections.append(([int(x1), int(y1), int(x2 - x1), int(y2 - y1)], box.conf[0].item(), CLASS_NAMES[int(box.cls[0].item())]))
                
                tracks = tracker.update_tracks(detections, frame=current_frame)
                current_lane_counts = {lane: 0 for lane in range(1, len(lane_polygons) + 1)}
                dashboard = np.zeros_like(current_frame, dtype=np.uint8)

                for track in tracks:
                    if not track.is_confirmed() or track.time_since_update > 1: continue
                    
                    track_id, bbox, class_name = track.track_id, track.to_ltrb(), track.get_det_class()
                    bottom_center = Point((bbox[0] + bbox[2]) / 2, bbox[3])
                    
                    track_history[track_id].append(bottom_center)
                    speed_kmph = -1
                    if len(track_history[track_id]) > int(fps / 2):
                        prev_pos = track_history[track_id][0]
                        pixel_dist = bottom_center.distance(prev_pos)
                        time_diff = len(track_history[track_id]) / fps
                        perspective_scale = 1 - (bottom_center.y / h)
                        real_world_dist = pixel_dist * perspective_scale * (REAL_WORLD_LANE_LENGTH_METERS / h)
                        speed_mps = real_world_dist / time_diff if time_diff > 0 else 0
                        speed_kmph = speed_mps * (3.6 if SPEED_UNIT == 'km/h' else 2.23694)
                    
                    assigned_lane = next((i + 1 for i, poly in enumerate(lane_polygons) if poly.contains(bottom_center)), None)
                    if assigned_lane: current_lane_counts[assigned_lane] += 1

                    # --- MODIFIED: Simplified Analysis and Visualization ---
                    is_stopped = False
                    
                    # Stopped vehicle check
                    if speed_kmph != -1 and speed_kmph < STOPPED_VEHICLE_SPEED_THRESHOLD_KMPH:
                        stopped_timers[track_id] += 1
                        if (stopped_timers[track_id] / fps) > STOPPED_VEHICLE_DURATION_SECONDS:
                            is_stopped = True
                            if not any(e['event'] == 'stopped_vehicle' and e['track_id'] == track_id for e in event_log):
                                event_log.append({'timestamp': current_frame_time, 'event': 'stopped_vehicle', 'track_id': track_id, 'lane': assigned_lane, 'duration_s': STOPPED_VEHICLE_DURATION_SECONDS})
                    else:
                        stopped_timers[track_id] = 0
                    
                    # --- REMOVED --- Wrong-way check has been completely removed.

                    # Set label and color based on stopped status
                    if is_stopped:
                        label = f"{class_name} {track_id} [STOPPED]"
                        color = (0, 0, 255) # Red for stopped
                    else:
                        label = f"{class_name} {track_id}"
                        if speed_kmph > 0:
                            label += f" {int(speed_kmph)} {SPEED_UNIT}"
                        color = (0, 255, 0) # Green for moving

                    cv2.rectangle(current_frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), color, 2)
                    cv2.putText(current_frame, label, (int(bbox[0]), int(bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                
                # Draw Dashboard and Lane Polygons
                y_offset = 40
                for lane_num, poly in enumerate(lane_polygons, 1):
                    is_congested = current_lane_counts[lane_num] > CONGESTION_THRESHOLD
                    lane_color = (0, 0, 255) if is_congested else (0, 255, 0)
                    poly_pts = np.array(poly.exterior.coords, np.int32).reshape((-1, 1, 2))
                    cv2.polylines(current_frame, [poly_pts], isClosed=True, color=lane_color, thickness=2)
                    status = "Congested" if is_congested else "Normal"
                    info_text = f"Lane {lane_num} ({status}): {current_lane_counts[lane_num]} veh"
                    cv2.putText(dashboard, info_text, (20, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
                    y_offset += 30
                    if is_congested and not any(e['event'] == 'congestion' and e['lane'] == lane_num and abs(e['timestamp'] - current_frame_time) < 1 for e in event_log):
                        event_log.append({'timestamp': current_frame_time, 'event': 'congestion', 'lane': lane_num, 'vehicle_count': current_lane_counts[lane_num]})
                
                final_frame = cv2.addWeighted(current_frame, 1, dashboard, 0.8, 0)
                out.write(final_frame)

            frames_batch = []
            elapsed_time = time.time() - start_time
            processed_frames = frame_num - len(frames_batch)
            fps_current = processed_frames / elapsed_time if elapsed_time > 0 else 0
            print(f"Processed frame {processed_frames}/{total_frames}... Current FPS: {fps_current:.2f}")

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    cap.release()
    out.release()
    print(f"\n--- Analysis Complete ---")
    print(f"Output video saved to: {OUTPUT_VIDEO_PATH}")
    if event_log:
        df = pd.DataFrame(event_log)
        df.to_csv(CSV_LOG_PATH, index=False)
        print(f"Event log saved to: {CSV_LOG_PATH}")
    else:
        print("No critical events were logged.")

In [ ]:
# CODE CELL 2: THE DEFINITIVE ANALYSIS SCRIPT

# --- 1. IMPORTS AND THE GUARANTEED PATH FIX ---
import sys
import os

# --- THE GUARANTEED FIX FOR THE IMPORT ERROR ---
# We temporarily change the working directory to the UFLD folder
# to make the import unambiguous. This is the most robust method.
original_cwd = os.getcwd()  # Save our current directory
ufld_path = '/kaggle/working/Ultra-Fast-Lane-Detection'
try:
    os.chdir(ufld_path)
    # This import will now definitely find the correct model.py
    from model import parsingNet
    print("✅ UFLD model definition imported successfully.")
finally:
    # IMPORTANT: Change back to the original directory so all other paths work correctly
    os.chdir(original_cwd)
# --- END OF FIX ---

import cv2
import torch
import numpy as np
import json
import time
from collections import deque, defaultdict
from shapely.geometry import Point, Polygon
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
print("✅ All other libraries imported successfully.")


# --- 2. CONFIGURATION ---
print("\n--- Loading Configuration ---")
YOLO_MODEL_PATH = 'yolov8l.pt'
LANE_MODEL_PATH = '/kaggle/input/ufld-tusimple-model/tusimple_18.pth'
INPUT_VIDEO_PATH = '/kaggle/input/test-video-data/test_vid.mp4' # <-- UPDATE THIS IF NEEDED

OUTPUT_DIR = '/kaggle/working/output_analysis'
os.makedirs(OUTPUT_DIR, exist_ok=True)
output_video_path = os.path.join(OUTPUT_DIR, f"analyzed_{os.path.basename(INPUT_VIDEO_PATH)}")
output_json_path = os.path.join(OUTPUT_DIR, f"analysis_log_{os.path.splitext(os.path.basename(INPUT_VIDEO_PATH))[0]}.json")
CONF_THRESHOLD = 0.40
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


# --- 3. UTILITY FUNCTIONS ---
def initialize_models():
    print("Initializing models...")
    vehicle_model = YOLO(YOLO_MODEL_PATH); vehicle_model.to(device)
    lane_model = parsingNet(pretrained=False, backbone='18', cls_dim=(201, 18, 4), use_aux=False).to(device)
    if not os.path.exists(LANE_MODEL_PATH):
        raise FileNotFoundError(f"UFLD model weights not found at {LANE_MODEL_PATH}. Did you add the Kaggle dataset?")
    state_dict = torch.load(LANE_MODEL_PATH, map_location='cpu')['model']
    lane_model.load_state_dict(state_dict); lane_model.eval()
    tracker = DeepSort(max_age=30, nn_budget=70, n_init=3)
    print("✅ All models initialized successfully.")
    return vehicle_model, lane_model, tracker

def process_frame_for_lanes(frame, lane_model):
    img_h, img_w, _ = frame.shape
    img_for_lane = cv2.resize(frame, (288, 800)); img_for_lane = cv2.cvtColor(img_for_lane, cv2.COLOR_BGR2RGB)
    img_for_lane = torch.from_numpy(img_for_lane.transpose((2, 0, 1))).float().unsqueeze(0).to(device) / 255.0
    with torch.no_grad(): out = lane_model(img_for_lane)
    col_sample = np.linspace(0, 288 - 1, 200); col_sample_w = col_sample[1] - col_sample[0]
    out_j = out[0].data.cpu().numpy()[:, ::-1, :]; prob = torch.softmax(torch.from_numpy(out_j), dim=0)
    idx = torch.arange(201, dtype=torch.float32).unsqueeze(1).unsqueeze(2); loc = torch.sum(prob * idx, dim=0)
    out_j = loc.numpy() * col_sample_w * img_w / 288
    lanes = []; row_anchor = np.linspace(590, 710, 18)
    for i in range(out_j.shape[1]):
        lane_points = out_j[:, i]; valid_points_mask = lane_points > 0
        if np.sum(valid_points_mask) > 2:
            lane = [];
            for k, is_valid in enumerate(valid_points_mask):
                if is_valid: lane.append((int(lane_points[k]), int(row_anchor[k] * img_h / 720.0)))
            if lane: lanes.append(lane)
    return lanes

def create_lane_polygons(lanes):
    if len(lanes) < 2: return []
    lanes = sorted([l for l in lanes if l], key=lambda l: l[0][0])
    if len(lanes) < 2: return []
    return [Polygon(lanes[i] + lanes[i+1][::-1]) for i in range(len(lanes) - 1)]

def get_vehicle_lane_assignment(bbox, lane_polygons):
    bottom_center = Point((bbox[0] + bbox[2]) / 2, bbox[3])
    for i, poly in enumerate(lane_polygons):
        if poly.contains(bottom_center): return i + 1
    return None

def draw_text_with_outline(img, text, origin, font=cv2.FONT_HERSHEY_SIMPLEX, scale=0.7, color=(255, 255, 255), thickness=2):
    (text_w, text_h), _ = cv2.getTextSize(text, font, scale, thickness)
    box_coords = ((origin[0] - 5, origin[1] + 5), (origin[0] + text_w + 5, origin[1] - text_h - 5))
    overlay = img.copy(); cv2.rectangle(overlay, box_coords[0], box_coords[1], (0, 0, 0), -1)
    img = cv2.addWeighted(overlay, 0.6, img, 0.4, 0)
    cv2.putText(img, text, origin, font, scale, color, thickness, cv2.LINE_AA)
    return img

# --- 4. MAIN PROCESSING ---
vehicle_model, lane_model, tracker = initialize_models()
cap = cv2.VideoCapture(INPUT_VIDEO_PATH);
if not cap.isOpened(): print(f"Error: Could not open video {INPUT_VIDEO_PATH}"); sys.exit(1)
w=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)); h=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)); fps=int(cap.get(cv2.CAP_PROP_FPS))
video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
frame_number = 0; full_analysis_log = {}; track_history = defaultdict(lambda: deque(maxlen=30)); lane_vehicle_ids = defaultdict(set);
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"\n--- Starting video analysis pipeline on {total_frames} frames ---"); start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read();
    if not ret: break
    frame_number += 1
    if frame_number % 50 == 0: print(f"Processing frame {frame_number}/{total_frames}...")

    detected_lanes = process_frame_for_lanes(frame, lane_model); lane_polygons = create_lane_polygons(detected_lanes)
    vehicle_results = vehicle_model(frame, conf=CONF_THRESHOLD, classes=[2, 3, 5, 7])
    detections_for_tracker = [[(det.xyxy[0].cpu().numpy()), det.conf[0].item(), int(det.cls[0].item())] for det in vehicle_results[0].boxes]
    detections_for_tracker = [([d[0][0], d[0][1], d[0][2]-d[0][0], d[0][3]-d[0][1]], d[1], d[2]) for d in detections_for_tracker]
    tracks = tracker.update_tracks(detections_for_tracker, frame=frame)

    current_frame_analysis = {'frame_id': frame_number, 'vehicles': [], 'lane_analysis': {}}; current_lane_counts = defaultdict(int)
    annotated_frame = frame.copy()
    
    for poly in lane_polygons:
        pts = np.array(poly.exterior.coords, np.int32).reshape((-1, 1, 2))
        overlay = annotated_frame.copy(); cv2.fillPoly(overlay, [pts], (0, 255, 255));
        annotated_frame = cv2.addWeighted(overlay, 0.15, annotated_frame, 0.85, 0)
        cv2.polylines(annotated_frame, [pts], isClosed=True, color=(255, 255, 0), thickness=2)

    for track in tracks:
        if not track.is_confirmed() or track.time_since_update > 2: continue
        track_id = track.track_id; ltrb = track.to_ltrb(); bbox = [int(v) for v in ltrb]
        class_id = track.get_det_class(); class_name = vehicle_model.names[class_id]
        center_point = ((bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2); track_history[track_id].append(center_point)
        speed_px_per_frame = 0;
        if len(track_history[track_id]) > 5:
            prev_point = track_history[track_id][0]; dx = center_point[0] - prev_point[0]; dy = center_point[1] - prev_point[1]
            speed_px_per_frame = np.sqrt(dx**2 + dy**2) / len(track_history[track_id])
        is_stopped = speed_px_per_frame < 1.0 and track.age > fps
        assigned_lane = get_vehicle_lane_assignment(bbox, lane_polygons)
        if assigned_lane: current_lane_counts[assigned_lane] += 1; lane_vehicle_ids[assigned_lane].add(track_id)
        
        color = (0, 0, 255) if is_stopped else (0, 255, 0); label = f"ID:{track_id} {class_name}"
        if assigned_lane: label += f" L:{assigned_lane}"
        cv2.rectangle(annotated_frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color, 2)
        annotated_frame = draw_text_with_outline(annotated_frame, label, (bbox[0], bbox[1] - 10), color=color)

    for i in range(len(lane_polygons)):
        lane_id = i + 1; num_vehicles = current_lane_counts.get(lane_id, 0)
        congestion = "High" if num_vehicles > 10 else "Medium" if num_vehicles > 5 else "Low"
        current_frame_analysis['lane_analysis'][f'lane_{lane_id}'] = {'vehicle_count':num_vehicles, 'total_vehicles_passed':len(lane_vehicle_ids[lane_id]), 'congestion_level':congestion}
    
    y_offset = 30
    annotated_frame = draw_text_with_outline(annotated_frame, f"Frame: {frame_number}", (10, y_offset)); y_offset += 35
    annotated_frame = draw_text_with_outline(annotated_frame, f"Total Vehicles: {len(tracks)}", (10, y_offset));
    for i in range(len(lane_polygons)):
        lane_id=i+1; lane_info=current_frame_analysis['lane_analysis'].get(f'lane_{lane_id}',{}); y_offset += 35
        text=f"Lane {lane_id}: {lane_info.get('vehicle_count',0)} ({lane_info.get('congestion_level','N/A')}) | Total: {lane_info.get('total_vehicles_passed',0)}";
        annotated_frame = draw_text_with_outline(annotated_frame, text, (10, y_offset))
        
    video_writer.write(annotated_frame);
    full_analysis_log[f'frame_{frame_number}'] = current_frame_analysis

cap.release(); video_writer.release(); end_time = time.time()
with open(output_json_path, 'w') as f: json.dump(full_analysis_log, f, indent=4)
print("\n--- ✅ Analysis Complete ---"); print(f"Processing time: {end_time - start_time:.2f} seconds")
print(f"Annotated video saved to: {os.path.abspath(output_video_path)}")
print(f"Detailed JSON log saved to: {os.path.abspath(output_json_path)}")

In [ ]:
# CODE CELL 2: SELF-CONTAINED TRAINING
# --- 1. INSTALL DEPENDENCIES FOR THIS CELL ---
!pip install ultralytics -q

# --- 2. IMPORT AND TRAIN ---
from ultralytics import YOLO

# Define paths
CUSTOM_MODEL_YAML_PATH = '/kaggle/working/yolov8l_dropout.yaml'
DATA_YAML_PATH = '/kaggle/working/vehicle_data.yaml'

print("Dependencies installed and import successful.")
print("Loading model from configuration...")
model = YOLO(CUSTOM_MODEL_YAML_PATH)

print("Starting training with Transfer Learning...")
results = model.train(
    data=DATA_YAML_PATH,
    pretrained='yolov8l.pt',
    imgsz=640,
    batch=8, # A smaller batch size is needed for larger models
    epochs=100,
    name='yolov8l_dropout_run',
    project='/kaggle/working/runs/train',
    cache=True,
    patience=30,
)

print(f"\n--- Training Complete ---")
# The path to the best model will be printed at the end of training.

In [ ]:
# CODE CELL 4: TRAINING
import sys
import os

# --- THE GUARANTEED FIX ---
ULTRALYTICS_PATH = '/kaggle/working/ultralytics'
if ULTRALYTICS_PATH not in sys.path:
    sys.path.insert(0, ULTRALYTICS_PATH)
print(f"Ensured '{ULTRALYTICS_PATH}' is at the front of sys.path.")

from ultralytics import YOLO

# Define the paths
CUSTOM_MODEL_YAML_PATH = '/kaggle/working/yolov8s_max_custom.yaml'
DATA_YAML_PATH = '/kaggle/working/vehicle_data.yaml'

print("Import successful. Loading model...")
model = YOLO(CUSTOM_MODEL_YAML_PATH)

print("Starting training...")
results = model.train(
    data=DATA_YAML_PATH,
    imgsz=640,
    batch=16,
    epochs=150,
    name='yolov8_multi_class_run',
    project='/kaggle/working/runs/train',
    cache=True,
    patience=50,
)

BEST_MODEL_PATH = '/kaggle/working/runs/train/yolov8_multi_class_run/weights/best.pt'
print(f"\n--- Training Complete ---")
print(f"Best model weights saved at: {BEST_MODEL_PATH}")

In [ ]:
# CODE CELL 4: ANALYSIS
import sys
import os
import cv2
import torch
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort
from shapely.geometry import Point, Polygon

# Add the sys.path fix here too, just in case you run this cell independently
ULTRALYTICS_PATH = '/kaggle/working/ultralytics'
if ULTRALYTICS_PATH not in sys.path:
    sys.path.insert(0, ULTRALYTICS_PATH)

from ultralytics import YOLO

# --- CONFIGURATION ---
BEST_MODEL_PATH = '/kaggle/working/runs/train/yolov8_multi_class_run/weights/best.pt'
FINAL_CLASS_NAMES = ['car', 'truck', 'bus', 'motorcycle']
VIDEO_PATH = '/kaggle/input/traffic-dataset/test_vid.mp4'
LANE_LINES = [[(100,720),(500,450)],[(550,720),(650,450)],[(800,720),(850,450)],[(1200,720),(1100,450)]]
OUTPUT_VIDEO_PATH = '/kaggle/working/final_multiclass_analysis.mp4'
CONF_THRESHOLD = 0.35

# --- INITIALIZE & PROCESS ---
device='cuda' if torch.cuda.is_available() else 'cpu'
model=YOLO(BEST_MODEL_PATH); model.to(device)
tracker=DeepSort(max_age=30,nn_budget=50,n_init=3)
cap=cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened(): print(f"Error opening video {VIDEO_PATH}")
else:
    w,h,fps=(int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH,cv2.CAP_PROP_FRAME_HEIGHT,cv2.CAP_PROP_FPS))
    out=cv2.VideoWriter(OUTPUT_VIDEO_PATH,cv2.VideoWriter_fourcc(*'mp4v'),fps,(w,h))
    lane_polygons=[Polygon(LANE_LINES[i]+LANE_LINES[i+1][::-1]) for i in range(len(LANE_LINES)-1)]
    lane_vehicle_ids={i+1:{cls:set() for cls in FINAL_CLASS_NAMES} for i in range(len(lane_polygons))}
    frame_num=0
    while cap.isOpened():
        ret,frame=cap.read();
        if not ret:break
        if frame_num%100==0:print(f"Processing frame {frame_num}/{int(cap.get(cv2.CAP_PROP_FRAME_COUNT))}...")
        frame_num+=1
        results=model(frame,verbose=False)
        detections=[]
        for box in results[0].boxes:
            if box.conf[0]>CONF_THRESHOLD:x1,y1,x2,y2=box.xyxy[0].cpu().numpy();detections.append(([int(x1),int(y1),int(x2-x1),int(y2-y1)],box.conf[0],int(box.cls[0])))
        tracks=tracker.update_tracks(detections,frame=frame)
        for track in tracks:
            if not track.is_confirmed() or track.time_since_update>2:continue
            track_id,bbox,cls_id=track.track_id,track.to_ltrb(orig_image_wh=(w,h)),track.get_det_class();class_name=FINAL_CLASS_NAMES[cls_id]
            assigned_lane=next((i+1 for i,poly in enumerate(lane_polygons) if poly.contains(Point((bbox[0]+bbox[2])/2,bbox[3]))),None)
            if assigned_lane:lane_vehicle_ids[assigned_lane][class_name].add(track_id)
            label=f"{class_name} {track_id}"+(f" L:{assigned_lane}" if assigned_lane else "");cv2.rectangle(frame,(int(bbox[0]),int(bbox[1])),(int(bbox[2]),int(bbox[3])),(0,255,0),2);cv2.putText(frame,label,(int(bbox[0]),int(bbox[1])-10),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,255,0),2)
        y_offset=40
        for lane_num,class_counts in lane_vehicle_ids.items():
            total_in_lane=sum(len(ids) for ids in class_counts.values());cv2.putText(frame,f"Lane {lane_num} Total: {total_in_lane}",(20,y_offset),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),3);y_offset+=30
            breakdown=", ".join([f"{name}:{len(ids)}" for name,ids in class_counts.items() if len(ids)>0]);cv2.putText(frame,breakdown,(30,y_offset),cv2.FONT_HERSHEY_SIMPLEX,0.6,(200,200,200),2);y_offset+=40
        out.write(frame)
    cap.release();out.release();print(f"\n--- Analysis Complete ---")

In [ ]:
# CODE CELL 5
import cv2
import torch
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from shapely.geometry import Point, Polygon

# Define the class names again after restart
FINAL_CLASS_NAMES = ['car', 'truck', 'bus', 'motorcycle']

# Read the best model path from the file
with open("/kaggle/working/best_model_path.txt", "r") as f:
    YOLO_MODEL_PATH = f.read().strip()

print(f"Using model from: {YOLO_MODEL_PATH}")

# --- ACTION REQUIRED: CONFIGURE THESE ---
VIDEO_PATH = '/kaggle/input/traffic-dataset/test_vid.mp4'
LANE_LINES = [[(100,720),(500,450)],[(550,720),(650,450)],[(800,720),(850,450)],[(1200,720),(1100,450)]]
# ----------------------------------------

CLASS_NAMES_FROM_TRAINING = FINAL_CLASS_NAMES
OUTPUT_VIDEO_PATH = '/kaggle/working/final_multiclass_analysis.mp4'
CONF_THRESHOLD = 0.35

# --- INITIALIZE MODELS ---
device='cuda' if torch.cuda.is_available() else 'cpu'
model=YOLO(YOLO_MODEL_PATH); model.to(device)
tracker=DeepSort(max_age=30,nn_budget=50,n_init=3)

# --- MAIN PROCESSING LOOP ---
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened(): print(f"Error: Could not open video {VIDEO_PATH}")
else:
    w,h,fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
    lane_polygons = [Polygon(lane_lines[i]+lane_lines[i+1][::-1]) for i in range(len(lane_lines)-1)]
    lane_vehicle_ids = {i+1: {cls: set() for cls in CLASS_NAMES_FROM_TRAINING} for i in range(len(lane_polygons))}

    frame_num = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break
        if frame_num % 100 == 0: print(f"Processing frame {frame_num}/{int(cap.get(cv2.CAP_PROP_FRAME_COUNT))}...")
        frame_num += 1

        results = model(frame, verbose=False)
        detections = []
        for box in results[0].boxes:
            if box.conf[0] > CONF_THRESHOLD:
                x1,y1,x2,y2=box.xyxy[0].cpu().numpy()
                detections.append(([int(x1),int(y1),int(x2-x1),int(y2-y1)],box.conf[0],int(box.cls[0])))
        
        tracks = tracker.update_tracks(detections, frame=frame)
        
        for track in tracks:
            if not track.is_confirmed() or track.time_since_update > 2: continue
            track_id, bbox, cls_id = track.track_id, track.to_ltrb(orig_image_wh=(w,h)), track.get_det_class()
            class_name = CLASS_NAMES_FROM_TRAINING[cls_id]
            assigned_lane = get_vehicle_lane(bbox, lane_polygons)
            if assigned_lane: lane_vehicle_ids[assigned_lane][class_name].add(track_id)
            label = f"{class_name} {track_id}" + (f" L:{assigned_lane}" if assigned_lane else "")
            cv2.rectangle(frame,(int(bbox[0]),int(bbox[1])),(int(bbox[2]),int(bbox[3])),(0,255,0),2)
            cv2.putText(frame,label,(int(bbox[0]),int(bbox[1])-10),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,255,0),2)
        
        y_offset = 40
        for lane_num, class_counts in lane_vehicle_ids.items():
            total_in_lane = sum(len(ids) for ids in class_counts.values())
            cv2.putText(frame,f"Lane {lane_num} Total: {total_in_lane}",(20,y_offset),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),3)
            y_offset += 30
            breakdown = ", ".join([f"{name}: {len(ids)}" for name,ids in class_counts.items() if len(ids)>0])
            cv2.putText(frame,breakdown,(30,y_offset),cv2.FONT_HERSHEY_SIMPLEX,0.6,(200,200,200),2)
            y_offset += 40
        out.write(frame)

    cap.release(); out.release()
    print(f"\n--- Analysis Complete ---")